In [1]:
import os

import pandas as pd
import autonormalize as an

In [2]:
pollution_df = pd.read_csv(os.path.join(os.getcwd(), 'autonormalize/downloads/pollution.csv'))
pollution_df = pollution_df.drop(columns=pollution_df.columns[9:])
pollution_df = pollution_df.drop(range(1000000, 1746661))
pollution_df = pollution_df.drop(columns=['Date Local'])
pollution_df = pollution_df.drop_duplicates()
print("Rows: "+ str(pollution_df.shape[0]))
print("Columns: " + str(pollution_df.shape[1]))
pollution_df.head(3)

Rows: 746021
Columns: 8


,Unnamed: 0,State Code,County Code,Site Num,Address,State,County,City
0,0,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix
1,1,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix
2,2,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix


In [11]:
deps_exact = an.find_dependencies(pollution_df, 1.00)
deps_approx = an.find_dependencies(pollution_df, 0.94)
print("Exact dependencies...")
print(deps_exact)
print("\nApproximate dependencies...")
print(deps_approx)

100%|██████████| 8/8 [04:40<00:00, 29.71s/it]


Exact dependencies...
 --> Unnamed: 0
 {County,Site Num}  {County,County Code}  {Address}  {State}  {County,City}  --> State Code
 {County,Site Num}  {County,State}  {County,State Code}  {Address}  {County,City}  --> County Code
 {Address}  --> Site Num
 {Site Num,County Code,State}  {County,Site Num}  {Site Num,State Code,County Code}  --> Address
 {County,Site Num}  {County,County Code}  {State Code}  {Address}  {County,City}  --> State
 {County Code,State}  {State Code,County Code}  {Address}  --> County
 {Site Num,County Code,State}  {County,Site Num}  {Address}  {Site Num,State Code,County Code}  --> City

Approximate dependencies...
 --> Unnamed: 0
 {County Code,Unnamed: 0}  {City}  {Site Num,County Code}  {Address}  {State}  {County}  --> State Code
 {Site Num,City}  {State Code,City}  {Address}  {State,City}  {County}  --> County Code
 {Address}  --> Site Num
 {Site Num,County Code}  {County,Site Num}  {Site Num,City}  --> Address
 {County Code,Unnamed: 0}  {City}  {State Code}

In [10]:
groupings = an.normalize_dependencies(deps_approx)
for grp in groupings:
    print('\n~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~\n')
    print(grp)


~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~

 --> Unnamed: 0
 {County Code,Unnamed: 0}  {City}  {Site Num,County Code}  {Address}  {State}  {County}  --> State Code
 {Site Num,State Code}  {City}  {Site Num,State}  {Address}  {County}  --> County Code
 {City}  {City}  {Address}  {City}  {City}  --> Site Num
 {Site Num,State Code}  {County,Site Num}  {City}  {City}  {City}  {Site Num,State}  {Site Num,County Code}  {City}  {City}  --> Address
 {County Code,Unnamed: 0}  {City}  {State Code}  {Site Num,County Code}  {Address}  {County}  --> State
 {Site Num,State Code}  {County Code,Unnamed: 0}  {State Code,County Code}  {City}  {County Code,State}  {Site Num,State}  {Site Num,County Code}  {Address}  --> County
 {Site Num,State Code}  {County,Site Num}  {Site Num,State}  {Site Num,County Code}  {Address}  --> City
